# Momentum Strategy

Sample momentum algorithm using polars.

Using daily data, apply these rules to find stocks ready to take off.

- close > 1.3 of the 52 week low
- close < .9 of the 52 week high 
- close > 50 day simple moving average (SMA)
- 50 day SMA> 150 day SMA
- 150 day SMA > 200 day SMA
- 200 day SMA > 200 day SMA from 1 month ago
- RSI 1 month ago >= 80

In [1]:
import polars as pl
import arrow
import talib
from glob import glob
from tqdm.notebook import tqdm

In [2]:
# use glob to grab the list of parquet files
daily_dir = "./data/daily"
file_list = glob(f"{daily_dir}/*.parquet")

In [3]:
def momentum_algo(filename, rsi_lookback=14):
    lf=(pl.scan_parquet(filename)
        
        # only need to calculate the last 52 weeks
        .filter(
            pl.col("Date") > arrow.now().shift(weeks=-52).datetime.replace(tzinfo=None)
        )
        .sort("Date")
        
        # some basic TA functions
        .with_columns(
            [
                pl.col("Adj Close").rolling_mean(window_size=200).alias("200 SMA"),
                pl.col("Adj Close").rolling_mean(window_size=150).alias("150 SMA"),
                pl.col("Adj Close").rolling_mean(window_size=50).alias("50 SMA"),
                
                # For RSI, we use the rolling_map function. The map returns a list
                # in the rolling period which is sent to the talib.RSI function.
                # Since we only need the last data value from the RSI computation, 
                # we just grab the last element.
                pl.col("Adj Close").rolling_map(
                    # the timeperiod for RSI is 1 less than the length of data.
                    lambda d: talib.RSI(d.to_numpy(), timeperiod=(len(d)-1))[-1],
                    window_size=rsi_lookback
                ).alias("RSI"),
                pl.col("High").max().alias("52 week high"),
                pl.col("Low").min().alias("52 week low"),
            ]
        )
        
        # get the 2 month old values
        .filter(
            pl.col("Date") > arrow.now().shift(months=-2).datetime.replace(tzinfo=None)
        )
        .with_columns(
            [
                pl.first("200 SMA").alias("200 SMA 2 month"),
                pl.first("150 SMA").alias("150 SMA 2 month"),
            ]
        )
          
        # get the 1 month old values
        .filter(
            pl.col("Date") > arrow.now().shift(months=-1).datetime.replace(tzinfo=None)
        )
    
        # add columns for the previous values
        .with_columns(
            [
                pl.first("200 SMA").alias("200 SMA 1 month"),
                pl.first("150 SMA").alias("150 SMA 1 month"),
                pl.first("50 SMA").alias("50 SMA 1 month"),
                pl.first("RSI").alias("RSI 1 month"),
            ]
        )
          
        # we only need the last column to process the conditions
        .last()
          
        # apply the conditions as new columns
        .with_columns(
            [
                (pl.col("Adj Close") > (1.3 * pl.col("52 week low"))).alias("condition 1"),
                (pl.col("Adj Close") < (0.9 * pl.col("52 week high"))).alias("condition 2"),
                (pl.col("Adj Close") > pl.col("50 SMA")).alias("condition 3"),
                (pl.col("50 SMA") > pl.col("150 SMA")).alias("condition 4"),
                (pl.col("150 SMA") > pl.col("200 SMA")).alias("condition 5"),
                (pl.col("200 SMA") > pl.col("200 SMA 1 month")).alias("condition 6"),
                (pl.col("RSI 1 month") >= 80.0).alias("condition 7"),
            ]
        )
          
        # selet the condition columns which are boolean types
        .select(
            pl.col(pl.Boolean)
        )
    )
    # collect the content (last row of boolean values) and use the all() function
    # on the list to test if the stock meets all of the criteria. i.e. if all the
    # values are true then the return value from all() is true.
    return all(list(lf.collect().row(0)))

In [4]:
def process_candidates():
    candidates = []
    for filename in tqdm(file_list, desc="Finding Momentum..."):
        symbol = filename.split("\\")[-1].split(".")[0]
        if momentum_algo(filename):
            candidates.append(symbol)
    return candidates

In [5]:
process_candidates()

Finding Momentum...:   0%|          | 0/3 [00:00<?, ?it/s]

AttributeError: 'Expr' object has no attribute 'rolling_map'